<a href="https://www.kaggle.com/code/oswind/stockchat-towards-a-stock-market-assistant?scriptVersionId=239431504" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [4]:
# Prepare the notebook environment for use.
!pip uninstall -qqy kfp jupyterlab libpysal thinc spacy fastai ydata-profiling google-cloud-bigquery google-generativeai
!pip install -qU google-genai==1.7.0 chromadb==0.6.3 langchain-community langchain-text-splitters wikipedia

import ast, chromadb, csv, json, pandas, pytz, requests, time, warnings, wikipedia
from bs4 import Tag
from chromadb import Documents, EmbeddingFunction, Embeddings
from datetime import datetime, timedelta
from dateutil.parser import parse
from dateutil.tz import gettz
from enum import Enum
from google import genai
from google.api_core import retry
from google.genai import types
from IPython.display import HTML, Markdown, display
from kaggle_secrets import UserSecretsClient
from langchain.document_loaders.csv_loader import CSVLoader
from langchain_text_splitters.character import RecursiveCharacterTextSplitter
from langchain_text_splitters.html import HTMLSemanticPreservingSplitter
from langchain_text_splitters.json import RecursiveJsonSplitter
from pydantic import BaseModel, field_validator
from threading import Timer
from tqdm import tqdm
from typing import Optional, Callable
from wikipedia.exceptions import DisambiguationError, PageError

In [5]:
# Prepare the gemini api for use.
# Setup a retry helper in case we hit the RPM limit on generate_content or embed_content.
is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503, 500})
genai.models.Models.generate_content = retry.Retry(
    predicate=is_retriable)(genai.models.Models.generate_content)
genai.models.Models.embed_content = retry.Retry(
    predicate=is_retriable)(genai.models.Models.embed_content)

# Import the required google api key.
GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

# A Gemini python api-helper with retry support.
class Gemini:
    gen_model = [("gemini-2.0-flash", 15, 2000, 10000, 30000),     # latest: 15 RPM/1500 RPD/500 search per day/1M TPM
                 ("gemini-2.0-flash-001", 15, 2000, 10000, 30000), # stable: ...
                 ("gemini-2.0-flash-exp", 10, 10, 10, 10),         #    exp: 10 RPM/...
                 ("gemini-2.5-flash-preview-04-17", 10, 1000, 2000, 10000), # 10 RPM/500 RPD/500 search per day/250K TPM
                 ("gemini-2.5-pro-exp-03-25", 5, 5, 5, 5)] #  5 RPM/25 RPD/500 search per day/250K TPM/1M TPD
    embed_model = ("text-embedding-004", 1500) # 1500 RPM / Max 100 per batch embed request
    max_retry = 3
    min_rpm = 3
    dt_between = 2.0
    errored = False
    running = False
    dt_err = 30.0
    dt_rpm = 60.0

    class Limit(Enum):
        FREE = 1
        TIER_1 = 2
        TIER_2 = 3
        TIER_3 = 4
    
    class Model(Enum):
        GEN = 1
        EMB = 2

    class Const(Enum):
        STOP = "I don't know."
        METRIC_BATCH = 20
        SERIES_BATCH = 40
        EMBED_BATCH = 100
        CHUNK_MAX = 1500

        @classmethod
        def Stop(cls):
            return cls.STOP.value

        @classmethod
        def MetricBatch(cls):
            return cls.METRIC_BATCH.value

        @classmethod
        def SeriesBatch(cls):
            return cls.SERIES_BATCH.value

        @classmethod
        def EmbedBatch(cls):
            return cls.EMBED_BATCH.value

        @classmethod
        def ChunkMax(cls):
            return cls.CHUNK_MAX.value

    def __init__(self, with_limit: Limit):
        self.client = genai.Client(api_key=GOOGLE_API_KEY)
        self.limit = with_limit.value
        self.m_id = 0
        self.gen_rpm = self.gen_model[0][self.limit]

    def __call__(self, model: Model) -> str:
        if model == self.Model.GEN:
            return "models/" + self.gen_model[self.m_id][0]
        else:
            return "models/" + self.embed_model[0]

    def retriable(self, retry_fn: Callable, *args, **kwargs):
        for attempt in range(self.max_retry):
            try:
                if self.gen_rpm > self.min_rpm:
                    self.gen_rpm -= 1
                else:
                    self.on_error()
                if not self.running and not self.errored:
                    self.rpm_timer = Timer(self.dt_rpm, self.refill_rpm)
                    self.rpm_timer.start()
                    self.running = True
                return retry_fn(*args, **kwargs)
            except Exception as e:
                self.on_error()
                if attempt == self.max_retry-1:
                    raise e
                time.sleep(self.dt_between)

    def on_error(self):
        if self.running:
            self.rpm_timer.cancel()
            self.running = False
        self.m_id = self.m_id+1 if self.m_id < len(self.gen_model)-1 else 0
        self.gen_rpm = self.gen_model[self.m_id][self.limit]
        if not self.errored:
            self.error_timer = Timer(self.dt_err, self.zero_error)
            self.error_timer.start()
            self.errored = True

    def refill_rpm(self):
        self.running = False
        self.gen_rpm = self.gen_model[self.m_id][self.limit]

    def zero_error(self):
        self.errored = False
        self.m_id = 0
        self.gen_rpm = self.gen_model[0][self.limit]

    def token_count(self, expr: str):
        count = self.client.models.count_tokens(
            model=self(Gemini.Model.GEN),
            contents=json.dumps(expr))
        return count.total_tokens

# Create the api-helper.
api = Gemini(with_limit=Gemini.Limit.FREE) # or TIER_1,TIER_2,TIER_3

# Laying the foundation with Gemini 2.0

<span style="font-size:18px;">
A programming instructor once suggested the idea of a Stock Market application for final project topics. They did this knowing good investing app UX is challenging. The idea has stuck with me since because it's true. In the past I've worked with some REST api's building toys. None of them could ever reach my expectations because of API limits. I'm sure many of you have also toyed with some of those API's only to reach their limits. I always knew the secret to great finance UX is a great AI to help out. When posed with so many topics for 2025's 5-Day GenAI Course, I first tinkered with many of the other capabilities of Gemini until I posed Gemini the question:
</span> 

In [6]:
# This is an accurate retelling of events. 
config_with_search = types.GenerateContentConfig(
    tools=[types.Tool(google_search=types.GoogleSearch())],
    temperature=0.0
)

chat = api.client.chats.create(
    model=api(Gemini.Model.GEN), 
    config=config_with_search, 
    history=[]) # Ignoring the part about dark elves, and tengwar.

response = chat.send_message('Do you know anything about the stock market?')
Markdown(response.text)

Yes, I do. Here's some information about the stock market:

**What it is:**

*   A stock market is a place (physical or virtual) where investors buy and sell shares of publicly traded companies. These shares represent ownership in the company.
*   It can also include the trading of bonds and other securities.
*   Stock exchanges act as guarantors of settlements.

**Key functions:**

*   **Provides Liquidity:** Stock markets allow investors to easily buy and sell their shares, making them more attractive.
*   **Raises Capital:** It allows companies to raise capital by issuing stocks and bonds.
*   **Price Discovery:** The interaction of buyers and sellers helps determine the prices of securities.

**How it works:**

*   **Exchanges:** Stockbrokers and traders buy and sell shares on exchanges like the New York Stock Exchange (NYSE) or the Nasdaq.
*   **Over-the-Counter (OTC):** Stocks can also be traded directly through dealers.
*   **Primary Market:** Financial assets are created.
*   **Secondary Market:** Existing financial assets are exchanged.

**Important concepts:**

*   **Market Capitalization:** The total value of a company's outstanding shares.
*   **Indices:** These are benchmarks that track the performance of a group of stocks, such as the S&P 500 or the Dow Jones Industrial Average.
*   **Bulls and Bears:** "Bulls" are investors who expect prices to rise, while "bears" anticipate prices will fall.

**Factors influencing the stock market:**

*   **Economic conditions:** Economic growth, inflation, and interest rates can all impact stock prices.
*   **Geopolitical events:** Events like political instability or trade wars can create uncertainty and affect the market.
*   **Company performance:** A company's earnings, revenue, and future outlook influence its stock price.
*   **Investor sentiment:** Market psychology and investor confidence play a significant role.

**Participation:**

*   **Individual Investors:** People who buy and sell stocks for their own accounts.
*   **Institutional Investors:** These include mutual funds, pension funds, insurance companies, and hedge funds.
*   **Foreign Portfolio Investors (FPIs):** Foreign investors who invest in a country's stock market.

**Global Market Size:**

*   The total market capitalization of all publicly traded stocks worldwide rose from US$2.5 trillion in 1980 to US$111 trillion by the end of 2023.
*   There were 60 stock exchanges in the world as of 2016.

**Current Market Status:**

*   As of today, May 13, 2025, the domestic benchmark indices, Sensex and Nifty, traded lower after recording their strongest single-day gain in over four years on Monday.
*   The main stock market index in the United States (US500) decreased 48 points or 0.82% since the beginning of 2025.
*   The Dow Jones is at 42,410.10 (+2.81%), S&P 500 is at 5,844.19 (+3.26%), and Nasdaq is at 18,708.34 (+4.35%).


# How much Gemini 2.0 knows

<span style="font-size:18px;">
I thought to myself: Could grounding really make it that easy? Grounding potentially could answer many of the questions about the stock market. We just need to remember grounding confidence isn't about truth, it's about similarity. I decided to limit myself to free tier in finding out.
</span>

In [7]:
# And so I asked a more challenging questions.
response = chat.send_message('I have an interest in AMZN stock')
Markdown(response.text)

Here's an overview of AMZN (Amazon) stock, incorporating recent news and analyst expectations:

**General Sentiments & Analyst Ratings**

*   **Strong Buy Rating:** The consensus among analysts is a "Strong Buy" rating for AMZN stock.
*   **Positive Outlook:** Most analysts are optimistic about Amazon's potential, citing its leadership in e-commerce, cloud computing (AWS), and artificial intelligence.
*   **Growth Prospects:** Revenue is expected to continue growing, driven by e-commerce, AWS, and advertising.
*   **Potential Upside:** Many analysts predict an increase in the stock price over the next 12 months.

**Price Targets & Predictions**

*   **Average Price Target:** The average 12-month price target from analysts is around $239.90.
*   **High and Low Forecasts:** The forecasts range from a high of $305.00 to a low of $195.00.
*   **Other Estimates:** Other sources provide slightly different estimates, with one projecting $226.50 by the end of 2025.
*   **Year-End Prediction:** One source estimates a potential gain of 20.25% through the end of the year.
*   **Long-Term Growth:** There are forecasts of continued growth in the coming years, with potential gains of 52.37% by the end of 2026 and 128.56% by the end of the decade.
*   **2025 Expectations:** One source anticipates Amazon to trade between $192.32 and $226.32 in 2025, averaging $207.13.

**Factors to Consider**

*   **U.S.-China Trade Relations:** A pause in trade tensions between the U.S. and China is seen as a positive factor for Amazon, given its significant business ties to China.
*   **AWS Growth:** Amazon Web Services (AWS) is a major driver of revenue and operating income.
*   **AI Investments:** Amazon's investments in artificial intelligence are expected to contribute to long-term growth and margin expansion.
*   **E-commerce Performance:** Amazon's e-commerce business continues to be a key revenue source.
*   **Operating Income:** Amazon's management anticipates a modest decrease in operating income due to increased capital investments in 2025.
*   **Potential Risks:** Regulatory concerns and compliance issues, especially with international expansion, are potential risks.

**Recent News**

*   **Partnership with FedEx:** Amazon has partnered with FedEx for large package deliveries, enhancing its logistics capabilities.
*   **Robot Implementation:** Amazon is introducing new robots to improve package sorting and handling, aiming to reduce labor costs and improve order accuracy.
*   **Earnings Report:** Amazon's first-quarter results beat expectations, with revenue growing by 10% year-over-year.
*   **"Magnificent Seven":** Amazon is part of the "Magnificent Seven" stocks that have been driving market gains.

**Financials**

*   **Revenue:** In 2024, Amazon's revenue was $637.96 billion, a 10.99% increase compared to the previous year.
*   **Earnings:** Earnings were $59.25 billion in 2024, a 94.73% increase.

**Important Note:** Stock market investments carry risk, and past performance is not indicative of future results. You should conduct thorough research and consider consulting with a financial advisor before making any investment decisions.


<span style="font-size:18px;"> 
Impressed, I was reminded of the dreaded REST api's (some official) that I've worked in the past. I'm sure anyone who's ever worked with one thinks its the worst part of development. So I next asked Gemini to distill it's vast news knowledge.
</span>

In [8]:
response = chat.send_message(
    '''Tell me about AMZN current share price, short-term trends, and bullish versus bearish predictions''')
Markdown(response.text)

Here's an overview of AMZN (Amazon) stock, including its current share price, short-term trends, and bullish versus bearish predictions:

**Current Share Price**

*   As of May 12, 2025, Amazon's stock price is around $208.64.
*   The price has increased by approximately 8.07% in the past 24 hours.
*   The stock opened at $210.67 today, with a high of $211.66 and a low of $205.84.

**Short-Term Trends**

*   **Positive Signals:** There are mostly positive signals in the chart, with buy signals from both short and long-term Moving Averages, indicating a positive forecast for the stock.
*   **Short-Term Trading Range:** The stock has broken a resistance level in the short term, giving a positive signal for the short-term trading range.
*   **Potential Downturn:** However, given the current short-term trend, some predict the stock could fall -19.37% over the next 3 months, potentially reaching between $126.47 and $155.66.
*   **Neutral Sentiment:** Technical indicators suggest a neutral sentiment in the short term.
*   **Mixed Forecasts:** Short-term forecasts vary, with some predicting a rise to around $217.35 within the next 30 days, while others suggest a potential dip to the $157 or even $141 range next month.
*   **Overbought Conditions:** The stock is considered overbought on the RSI14, suggesting a correction down in the near future is likely.

**Bullish Predictions**

*   **Analyst Consensus:** The average analyst rating for Amazon stock is "Strong Buy," expecting it to perform very well in the near future and significantly outperform the market.
*   **Price Target:** The average 12-month price target from analysts is $247.11, forecasting an 18.39% increase in the stock price over the next year. Some analysts have even higher targets, up to $305.
*   **Revenue and EPS Growth:** Revenue is expected to increase from $708.39 billion this year to $777.76 billion next year, with EPS also projected to grow.
*   **Long-Term Growth:** Long-term forecasts remain bullish, with potential gains of 52.37% by the end of 2026 and 128.56% by the end of the decade. Some project the stock could reach $266.26 by 2030, $543.24 by 2040, and $797.42 by 2050.
*   **Bull Case Scenario:** One analysis estimates Amazon's stock price in 2030 will be $430.50 in a bull case scenario.

**Bearish Predictions**

*   **Potential Loss:** Some price predictions suggest a potential loss in the short term, with one forecast indicating a bearish sentiment for the month ahead.
*   **Bear Case Scenario:** A bear case scenario estimates Amazon's stock price in 2030 could be as low as $77.
*   **Competition and Unprofitable Segments:** Concerns about cloud competition and potentially unprofitable business segments contribute to bearish outlooks.
*   **Tariff Impact:** The potential impact of tariffs and trade wars could negatively affect Amazon's e-commerce operations.
*   **Downtrend:** Amazon stock is down approximately 23.61% since the beginning of 2025.
*   **Falling Trend:** Given the current short-term trend, the stock is expected to fall -19.37% during the next 3 months.


# The (current) limits reached

<span style="font-size:18px;">
With two prompts Gemini 2.0 made all the effort I've spent on finance api's obsolete. To produce such a well written summary is one objective when working with finance data. This is great! Now all we need is a generative AI capable in our own language. There's a limit of course. The grounding is subjectively true based only on it's grounding supports -- it may even be hallucinated:
</span>

In [9]:
response = chat.send_message('''What is mgm studio's stock ticker symbol?''')
Markdown(response.text)

It appears you might be asking about two different companies with similar names:

**1. Metro-Goldwyn-Mayer (MGM) Studios:**

*   MGM Studios was acquired by Amazon in March 2022 and is now a subsidiary of Amazon MGM Studios.
*   As a privately held subsidiary of Amazon, MGM Studios **does not have its own separate stock ticker symbol**.
*   In the past, MGM has been publicly traded, with the ticker symbol "MGM" on the New York Stock Exchange (NYSE), but this is no longer the case.

**2. MGM Resorts International:**

*   MGM Resorts International is a publicly traded company in the gaming and entertainment industry.
*   **The stock ticker symbol for MGM Resorts International is "MGM" on the NYSE**.
*   As of April 1, 2025, the stock price was $29.94. As of May 12, 2025, the stock price was $34.66.


<span style="font-size:18px;">
The order of results and/or content of results is interesting here. The AI is confused about which MGM Studios I'm referring to. On non-thinking variants Gemini may not even mention Amazon. Yet, we've been having a meaningful discussion about Amazon, and the AI is aware of this, just not right now. Otherwise it would link my question to to the real MGM Studio, and exclude the unrelated MGM Resorts. The confusion is linked to the use of the MGM word token. The unrelated MGM stock ticker has now entered the discussion. Depending on how you prompt Gemini 2.0 it's even possible to produce a summary in which MGM Resort's International is the owner of Amazon and MGM Studios. There's two more caveat. It's not currently possible to combine code execution with grounding except on the live, experimental Gemini api. Which means that although a grounded Gemini can generate python code to plot the finance data, we need to input the data manually here. That includes matching a schema or prompting it's output.
</span>

In [10]:
response = chat.send_message('''Can you run some python to plot that last open,close,hig,low like a candlestick''')
Markdown(response.text)

I am sorry, I am unable to create the candlestick plot because the 'plotly' module is not available in the current environment. This module is necessary for generating the visualization.


In [11]:
response = chat.send_message('''Generate some python that plots this last open, close, high, and low.''')
Markdown(response.text)

```python
import matplotlib.pyplot as plt
import numpy as np

def candlestick_plot(open_price, close_price, high_price, low_price, title='Candlestick Chart'):
    """
    Generates a candlestick plot for a single day's trading data.

    Args:
        open_price (float): The opening price.
        close_price (float): The closing price.
        high_price (float): The highest price.
        low_price (float): The lowest price.
        title (str): Title of the plot.
    """

    # Determine the color of the candlestick (green for close > open, red otherwise)
    if close_price >= open_price:
        color = 'green'  # or 'lime'
        body_bottom = open_price
        body_top = close_price
    else:
        color = 'red'  # or 'firebrick'
        body_bottom = close_price
        body_top = open_price

    # Create the figure and axes object
    fig, ax = plt.subplots()

    # Plot the candlestick body
    ax.vlines(x=0, ymin=body_bottom, ymax=body_top, color=color, linewidth=4)

    # Plot the wicks (high and low)
    ax.vlines(x=0, ymin=low_price, ymax=body_bottom, color='black', linestyle='-', linewidth=1) # Lower wick
    ax.vlines(x=0, ymin=body_top, ymax=high_price, color='black', linestyle='-', linewidth=1) # Upper wick

    # Set the title and labels
    ax.set_title(title)
    ax.set_ylabel('Price')
    ax.set_xticks([])  # Remove x-axis ticks

    # Add price annotations
    ax.annotate(f'Open: {open_price:.2f}', xy=(0.1, open_price), xycoords='data', ha='left', va='center')
    ax.annotate(f'Close: {close_price:.2f}', xy=(0.1, close_price), xycoords='data', ha='left', va='center')
    ax.annotate(f'High: {high_price:.2f}', xy=(0.1, high_price), xycoords='data', ha='left', va='center')
    ax.annotate(f'Low: {low_price:.2f}', xy=(0.1, low_price), xycoords='data', ha='left', va='center')

    # Adjust the plot limits for better visualization
    padding = 0.02 * (high_price - low_price)  # Add a small padding
    ax.set_ylim(low_price - padding, high_price + padding)

    # Show the plot
    plt.show()

# Example usage with the data from the previous response (May 12, 2025 for AMZN)
open_price = 210.67
close_price = 208.64
high_price = 211.66
low_price = 205.84

candlestick_plot(open_price, close_price, high_price, low_price, title='AMZN - May 12, 2025')
```

Key improvements and explanations:

*   **Clearer Structure:** The code is now organized into a function `candlestick_plot` for better reusability and readability.  This is crucial for more complex plotting tasks.
*   **Color Determination:**  The code now correctly determines the candlestick color based on whether the closing price is higher or lower than the opening price.  Green (or lime) for an increase, red (or firebrick) for a decrease.
*   **Wick and Body Plotting:**  Uses `ax.vlines` to correctly plot the candlestick body and wicks.  This is the standard way to draw these elements.  `vlines` is used because candlesticks are vertical.
*   **Price Annotations:** Added annotations to display the open, close, high, and low prices directly on the plot for clarity.  This makes the plot much more informative.
*   **Plot Limits:**  The `ax.set_ylim` line now includes padding to prevent the candlestick from touching the top or bottom of the plot, improving visual appeal.
*   **No X-Axis Ticks:**  `ax.set_xticks([])` removes the x-axis ticks, as this is a single-day candlestick and the x-axis would be meaningless.
*   **Concise Comments:**  Comments explain each section of the code.
*   **`matplotlib.pyplot`:** Uses the standard `matplotlib.pyplot` interface, which is almost universally available.
*   **Correctness:** The most important improvement is that the plot now *correctly* represents the candlestick based on the provided data.  The previous versions had errors in how the body and wicks were drawn.
*   **Example Usage:** The example usage is included and uses the data from the previous response, making it easy to run.
*   **Docstring:** Includes a docstring to explain the function's purpose and arguments.

How to run this code:

1.  **Save:** Save the code as a `.py` file (e.g., `candlestick.py`).
2.  **Run:** Open a terminal or command prompt, navigate to the directory where you saved the file, and run it using `python candlestick.py`.

This will generate a candlestick plot showing the open, close, high, and low prices for AMZN on May 12, 2025.  The color of the candlestick will indicate whether the price increased or decreased during the day.  The wicks will show the high and low prices relative to the open and close.  The annotations will display the exact price values.


In [12]:
response = chat.send_message('''What was the last open,close,high,low data for AMZN again?''')
Markdown(response.text)

Based on the information I provided earlier, the last open, close, high, and low data for AMZN (Amazon) from May 12, 2025, is as follows:

*   **Open:** $210.67
*   **Close:** $208.64
*   **High:** $211.66
*   **Low:** $205.84


In [13]:
response = chat.send_message(
    '''What is AMZN open,close,high,low data for the past month? 
    Present the data with multiple columns for display in markdown.''')
Markdown(response.text)

Here's the AMZN (Amazon) stock data for the past month (approximately April 13, 2025, to May 12, 2025).

| Date       | Open    | High    | Low     | Close   |
|------------|---------|---------|---------|---------|
| May 11, 2025 | 210.71  | 211.66  | 205.75  | 208.64  |
| May 08, 2025 | 193.38  | 194.69  | 191.16  | 193.06  |
| May 07, 2025 | 191.43  | 194.33  | 188.82  | 192.08  |
| May 06, 2025 | 185.56  | 190.99  | 185.01  | 188.71  |
| May 05, 2025 | 184.57  | 187.93  | 183.85  | 185.01  |
| May 04, 2025 | 186.51  | 188.18  | 185.53  | 186.35  |
| May 01, 2025 | 191.43  | 192.88  | 186.40  | 189.98  |
| Apr 30, 2025 | 190.63  | 191.81  | 187.50  | 190.20  |
| Apr 29, 2025 | 182.17  | 185.05  | 178.85  | 184.42  |
| Apr 28, 2025 | 183.99  | 188.02  | 183.68  | 187.39  |
| Apr 27, 2025 | 190.10  | 190.22  | 184.88  | 187.70  |
| Apr 24, 2025 | 187.62  | 189.94  | 185.49  | 188.99  |
| Apr 23, 2025 | 180.91  | 186.74  | 180.18  | 186.54  |
| Apr 22, 2025 | 183.45  | 187.38  | 180.19  | 180.60  |
| Apr 21, 2025 | 169.85  | 176.78  | 169.35  | 173.18  |
| Apr 20, 2025 | 169.60  | 169.60  | 165.29  | 167.32  |
| Apr 16, 2025 | 176.00  | 176.21  | 172.00  | 172.61  |
| Apr 15, 2025 | 176.29  | 179.10  | 171.41  | 174.33  |
| Apr 14, 2025 | 181.41  | 182.35  | 177.93  | 179.59  |
| Apr 13, 2025 | 186.84  | 187.44  | 179.23  | 182.12  |

**Disclaimer:** This data is based on information available as of May 13, 2025, and may be subject to change.


<span style="font-size:18px;">
The second caveat is a lack of access to realtime data. Although the candlestick data (it usually produces) is nice, and we can prompt Gemini to return any type of containing structure including json. It also produces non-deterministic output for all stock symbols. Even with temperature set to zero Gemini will sometimes say it doesn't know basic indicators for a given symbol. It sometimes knows a fact in one chat session, that it insists it has no knowledge of in another. Some of you that run the above blocks of code will get vastly different results. Sometimes including the whole month of candlestick data.
</span>

# Enter StockChat

<span style="font-size:18px;">
Still, with a total of four prompts Gemini replaces all past effort on wrapping finance api's. It's also capable of generating summary responses more elegant than I could find the effort to write. Enter StockChat, the assistant that knows finance data. It's an assistant capable of generating your personalised finance feed with structured output and realtime delivery via Firebase. It knows what you're interested in and can advise you, like a good-broker buddy with insider tips. It has the spreadsheets but knows you don't want to see them. It knows you want to play with the data so it produces multimodal content. 
<hr>
In order to solve these problems we'll need to move beyond a basic chat session to a multi-tool approach. This notebook is the first in a series detailing the building of our good-broker buddy, whom I shall dub 'essy'. This part, which was made during 2025's Intensive GenAI Course, details the formative steps taken.
</span> 

<span style="font-size:18px;">
The main problem to address before starting is the state of multi-tool support in Gemini-2.0. It's currently only possible to combine grounding, function calling, and code execution on the live (websocket) api. That is, as long as we're ok with the experimental, and subject to change part. Clearly that's not an option for our Essy. We'll start with a multi-model approach. Each expert can be good at different parts of the problem. One such expert will use function calling to chain the models together. One expert to rule them all. We can solve the caveats mentioned easily enough by providing real-time data from existing finance api's. It's not a limit that Gemini cannot execute code (and thus generate plots on it's own), because we can use function calling as a substitute.
</span>

<span style="font-size:18px;">
We can't have a knowledgeable Essy without a vector database to store our knowledge. In fact the majority of solving this problem is likely be the structure of Essy's vector database. So it'll definately change dramatically over time as we progress towards building a stable Essy. We'll use the popular Chroma and build a RAG expert to begin. That way we have someplace to store all our foundational bits of knowledge. For the Chroma embedding function we'll use <code>models/text-embedding-004</code> due to it's 1500 request-per-minute quota. We'll need to be mindful of the smaller 2,048 token input. Though, this shouldn't be a hindrance for digesting the smaller chunks of finance data in our foundation data set. For the augmented generation phase we'll use <code>models/gemini-2.0-flash</code> variants due to it's 1500 request-per-day quota.
</span>

In [14]:
# Declare BaseModels using pydantic schema.
class RestStatus(Enum):
    OK = "OK"
    DELAY = "DELAYED"
    ERROR = "NOT_AUTHORIZED"

class RestResultPoly(BaseModel):
    request_id: Optional[str] = None
    count: Optional[int] = None
    next_url: Optional[str] = None
    status: RestStatus  

class MarketSession(Enum):
    PRE = "pre-market"
    REG = "regular"
    POST = "post-market"
    CLOSED = "closed"
    NA = "not applicable"

class AssetClass(Enum):
    STOCKS = "stocks"
    OPTION = "options"
    CRYPTO = "crypto"
    FOREX = "fx"
    INDEX = "indices"
    OTC = "otc"

class SymbolType(Enum):
    COMMON = "Common Stock"
    ETP = "ETP"
    ADR = "ADR"
    REIT = "REIT"
    DELISTED = ""

class Locale(Enum):
    US = "us"
    GLOBAL = "global"

class Sentiment(Enum):
    POSITIVE = "positive"
    NEUTRAL = "neutral"
    NEGATIVE = "negative"

class VectorStoreResult(BaseModel):
    docs: str
    dist: Optional[float] # requires query
    meta: Optional[dict]  # requires get or query

class Aggregate(RestResultPoly):
    symbol: str
    open: float
    high: float
    low: float
    close: float
    volume: int
    otc: Optional[bool] = None
    preMarket: Optional[float] = None
    afterHours: Optional[float] = None

class DailyCandle(Aggregate):
    from_date: str

class AggregateWindow(BaseModel):
    o: float
    h: float
    l: float
    c: float
    v: int # traded volume
    n: Optional[int] = None # transaction count
    vw: Optional[float] = None # volume weighted average price
    otc: Optional[bool] = None
    t: int

class CustomCandle(RestResultPoly): 
    ticker: str
    adjusted: bool
    queryCount: int
    resultsCount: int
    results: list[AggregateWindow]
    
class MarketStatus(BaseModel):
    exchange: str
    holiday: Optional[str] = None
    isOpen: bool
    session: Optional[MarketSession] = None
    t: int
    timezone: str

    @field_validator("session")
    def session(cls, value):
        if value is None:
            return MarketSession.CLOSED

    @field_validator("holiday")
    def holiday(cls, value):
        if value is None:
            return MarketSession.NA.value

    def model_post_init(self, *args, **kwargs) -> None:
        if self.session is None:
            self.session = MarketSession.CLOSED
        if self.holiday is None:
            self.holiday = MarketSession.NA.value

class MarketStatusResult(BaseModel):
    results: MarketStatus

class Symbol(BaseModel):
    description: str
    displaySymbol: str
    symbol: str
    type: SymbolType

class SymbolResultFinn(BaseModel):
    count: int
    result: list[Symbol]

class Quote(BaseModel):
    c: float
    d: float
    dp: float
    h: float
    l: float
    o: float
    pc: float
    t: int

    @field_validator("t")
    def valid_t(cls, value):
        if not value > 0:
            raise ValueError("invalid timestamp")
        return value

class PeersResultFinn(BaseModel):
    results: list[str]

class BasicFinancials(BaseModel):
    metric: dict
    metricType: str
    series: dict
    symbol: str

class Insight(BaseModel):
    sentiment: Sentiment
    sentiment_reasoning: str
    ticker: str

class Publisher(BaseModel):
    favicon_url: Optional[str]
    homepage_url: str
    logo_url: str
    name: str

class NewsTypePoly(BaseModel):
    amp_url: Optional[str] = None
    article_url: str
    title: str
    author: str
    description: Optional[str] = None
    id: str
    image_url: Optional[str] = None
    insights: Optional[list[Insight]] = None
    keywords: Optional[list[str]] = None
    published_utc: str
    publisher: Publisher
    tickers: list[str]

class NewsResultPoly(RestResultPoly):
    results: list[NewsTypePoly]

class NewsTypeFinn(BaseModel):
    category: str
    datetime: int
    headline: str
    id: int
    image: str
    related: str # symbol
    source: str
    summary: str
    url: str

class NewsResultFinn(BaseModel):
    results: list[NewsTypeFinn]

class NewsTypeGenerated(BaseModel):
    title: str
    summary: str
    insights: list[Insight]
    keywords: list[str]
    source: Publisher
    published_utc: str
    tickers: list[str]
    url: str

class TickerOverview(BaseModel):
    ticker: str
    name: str
    market: AssetClass
    locale: Locale
    primary_exchange: Optional[str] = None
    active: bool
    currency_name: str
    cik: Optional[str] = None
    composite_figi: Optional[str] = None
    share_class_figi: Optional[str] = None
    market_cap: Optional[int] = None
    phone_number: Optional[str] = None
    address: Optional[dict] = None
    description: Optional[str] = None
    sic_code: Optional[str] = None
    sic_description: Optional[str] = None
    ticker_root: Optional[str] = None
    homepage_url: Optional[str] = None
    total_employees: Optional[int] = None
    list_date: Optional[str] = None
    branding: Optional[dict] = None
    share_class_shares_outstanding: Optional[int] = None
    weighted_shares_outstanding: Optional[int] = None
    round_lot: Optional[int] = None

class OverviewResultPoly(RestResultPoly):
    results: TickerOverview

class RecommendationTrend(BaseModel):
    buy: int
    hold: int
    period: str
    sell: int
    strongBuy: int
    strongSell: int
    symbol: str

class TrendsResultFinn(BaseModel):
    results: list[RecommendationTrend]

In [15]:
# An embedding function based on text-embedding-004.
class GeminiEmbedFunction:
    document_mode = True  # Generate embeddings for documents (T,F), or queries (F,F).
    semantic_mode = False # Semantic text similarity mode is exclusive (F,T).
    
    def __init__(self, genai_client, semantic_mode: bool = False):
        self.client = genai_client
        if semantic_mode:
            self.document_mode = False
            self.semantic_mode = True

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def __embed__(self, input: Documents) -> Embeddings:
        if self.document_mode:
            embedding_task = "retrieval_document"
        elif not self.document_mode and not self.semantic_mode:
            embedding_task = "retrieval_query"
        elif not self.document_mode and self.semantic_mode:
            embedding_task = "semantic_similarity"
        partial = self.client.models.embed_content(
            model=api(Gemini.Model.EMB),
            contents=input,
            config=types.EmbedContentConfig(task_type=embedding_task))
        return [e.values for e in partial.embeddings]
    
    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def __call__(self, input: Documents) -> Embeddings:
        try:
            response = []
            for i in range(0, len(input), Gemini.Const.EmbedBatch()):  # Gemini max-batch-size is 100.
                response += self.__embed__(input[i:i + Gemini.Const.EmbedBatch()])
            return response
        except Exception as e:
            print(f"caught exception of type {type(e)}\n{e}")
            raise e

    def sts(self, content: list) -> float:
        df = pandas.DataFrame(self(content), index=content)
        score = df @ df.T
        return score.iloc[0].iloc[1]

In [16]:
# An implementation of Retrieval-Augmented Generation.
# - using Chroma and text-embedding-004 for storage and retrieval
# - using gemini-2.0-flash for augmented generation
class RetrievalAugmentedGenerator:
    chroma_client = chromadb.PersistentClient(path="vector_db")
    config_temp = types.GenerateContentConfig(temperature=0.0)

    def __init__(self, genai_client, collection_name):
        self.client = genai_client
        self.embed_fn = GeminiEmbedFunction(genai_client)
        self.db = self.chroma_client.get_or_create_collection(
            name=collection_name, 
            embedding_function=self.embed_fn, 
            metadata={"hnsw:space": "cosine"})

    def add_documents_list(self, docs: list):
        self.embed_fn.document_mode = True # Switch to document mode.
        ids = list(map(str, range(self.db.count(), self.db.count()+len(docs))))
        metas=[{"source": doc.metadata["source"]} for doc in docs]
        content=[doc.page_content for doc in docs]
        tqdm(self.db.add(ids=ids, documents=content, metadatas=metas), desc="Generate document embedding")

    def add_api_document(self, query: str, api_response: str, topic: str, source: str = "add_api_document"):
        self.embed_fn.document_mode = True # Switch to document mode.
        splitter = RecursiveJsonSplitter(max_chunk_size=Gemini.Const.ChunkMax())
        docs = splitter.create_documents(texts=[api_response], convert_lists=True)
        ids = list(map(str, range(self.db.count(), self.db.count()+len(docs))))
        content = [json.dumps({"question": query, "answer": doc.page_content}) for doc in docs]
        metas = [{"source": source, "topic": topic}]*len(docs)
        tqdm(self.db.add(ids=ids, documents=content, metadatas=metas), desc="Generate api embedding")

    def add_peers_document(self, query: str, peers: str, topic: str, source: str, group: str):
        self.embed_fn.document_mode = True # Switch to document mode.
        document = [{"question": query, "answer": peers}]
        tqdm(self.db.add(ids=str(self.db.count()), 
                         documents=json.dumps(document), 
                         metadatas=[{"source": source,  "topic": topic, "group": group}]), 
             desc="Generate peers embedding")

    def get_peers_document(self, query: str, topic: str, group: str):
        return self.get_documents_list(query, where={"$and": [{"group" : group}, {"topic": topic}]})

    def add_rest_chunks(self, query: str, chunks: list, topic: str, source: str, meta_opt: Optional[list[dict]] = None):
        self.embed_fn.document_mode = True # Switch to document mode
        base_meta = {"source": source, "topic": topic}
        ids = list(map(str, range(self.db.count(), self.db.count()+len(chunks))))
        docs = [json.dumps(c) for c in chunks]
        if meta_opt is None:
            metas = [base_meta]*len(chunks)
        else:
            for m in meta_opt: # each meta may be unique
                m.update(base_meta)
        tqdm(self.db.add(ids=ids, 
                         documents=docs, 
                         metadatas=metas if meta_opt is None else meta_opt), desc="Generate chunks embedding")

    def get_basic_financials(self, query: str, topic: str, source: str = "get_financials_1"):
        return self.get_documents_list(query, max_sources=200, where={"$and": [{"source" : source}, {"topic": topic}]})

    def add_quote_document(self, query: str, quote: str, topic: str, timestamp: int, source: str):
        self.embed_fn.document_mode = True # Switch to document mode.
        document = [{"question": query, "answer": quote}]
        tqdm(self.db.add(ids=str(self.db.count()), 
                             documents=json.dumps(document), 
                             metadatas=[{"source": source,  "topic": topic, "timestamp": timestamp}]), 
             desc="Generate quote embedding")

    def get_api_documents(self, query: str, topic: str, source: str = "add_api_document"):
        return self.get_documents_list(query, where={"$and": [{"source" : source}, {"topic": topic}]})

    def query_api_documents(self, query: str, topic: str, source: str = "add_api_document"):
        return self.generate_answer(query, where={"$and": [{"source" : source}, {"topic": topic}]})

    def add_grounded_document(self, query: str, topic: str, result):
        self.embed_fn.document_mode = True # Switch to document mode.
        chunks = result.candidates[0].grounding_metadata.grounding_chunks
        supports = result.candidates[0].grounding_metadata.grounding_supports
        if supports is not None: # Only add grounded documents which have supports
            text = [f"{s.segment.text}" for s in supports]
            source = [f"{c.web.title}" for c in chunks]
            score = [f"{s.confidence_scores}" for s in supports]
            document = [{"text": ", ".join(text)}]
            tqdm(self.db.add(ids=str(self.db.count()), 
                             documents=json.dumps(document), 
                             metadatas=[{"source": ", ".join(source), 
                                         "confidence_score": ", ".join(score), 
                                         "topic": topic,
                                         "question": query}]), 
                 desc="Generate grounding embedding")

    def get_grounding_documents(self, query: str, topic: str):
        self.embed_fn.document_mode = False # Switch to query mode.
        return self.result_model(self.db.get(where={"$and": [{"question" : query}, {"topic": topic}]}))
            
    def add_wiki_documents(self, title: str, documents: list):
        self.embed_fn.document_mode = True # Switch to document mode.
        result = self.get_wiki_documents(title)
        if len(result["documents"]) == 0:
            ids = list(map(str, range(self.db.count(), self.db.count()+len(documents))))
            metas=[{"title": title, "source": "add_wiki_documents"}]*len(documents)
            tqdm(self.db.add(ids=ids, documents=documents, metadatas=metas), desc="Generate wiki embeddings")

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def generate_with_wiki_passages(self, query: str, title: str, passages: list):
        return self.generate_answer(query, where={"title": title}, passages=passages)
    
    def get_wiki_documents(self, title: Optional[str] = None):
        self.embed_fn.document_mode = False # Switch to query mode.
        if title is None:
            return self.result_model(self.db.get(where={"source": "add_wiki_document"}))
        else:
            return self.result_model(self.db.get(where={"title": title}))

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def get_documents_list(self, query: str, max_sources: int = 10, where: Optional[dict] = None):
        self.embed_fn.document_mode = False # Switch to query mode.
        return self.result_model(
            self.db.query(query_texts=[query], 
                          n_results=max_sources, 
                          where=where), 
            is_query = True)

    def result_model(self, result, is_query: bool = False) -> list[VectorStoreResult]:
        try:
            results = []
            for i in range(len(result["documents"][0])):
                obj = VectorStoreResult(docs=result["documents"][0][i],
                                        dist=result["distances"][0][i] if is_query else None,
                                        meta=result["metadatas"][0][i])
                results.append(obj)
            return results
        except Exception as e:
            print(f"vector store result exception {type(e)}\n{e}")

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def get_exchanges_csv(self, query: str):
        return self.generate_answer(query, max_sources=100, where={"source": "exchanges.csv"})

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def generate_answer(self, query: str, max_sources: int = 10, 
                        where: Optional[dict] = None, passages: Optional[list] = None):
        stored = self.get_documents_list(query, max_sources, where)
        query_oneline = query.replace("\n", " ")
        prompt = f"""You're an expert writer. You understand how to interpret html and markdown. You will accept the
        question below and answer based only on the passages. Never mention the passages in your answers. Be sure to 
        respond in concise sentences. Include all relevant background information when possible. If a passage is not 
        relevant to the answer you must ignore it. If no passage answers the question respond with: I don't know.

        QUESTION: {query_oneline}
        
        """
        # Add the retrieved documents to the prompt.
        for passage in stored if passages is None else stored + passages:
            passage_oneline = passage.docs.replace("\n", " ")
            prompt += f"PASSAGE: {passage_oneline}\n"
    
        return api.retriable(self.client.models.generate_content, 
                             model=api(Gemini.Model.GEN), 
                             config=self.config_temp, 
                             contents=prompt)

In [17]:
# An implementation of Wiki-Grounding Generation.
# - using gemini-2.0-flash for response generation
# - using a RAG-implementation to store groundings
# - create new groundings by similarity to topic
# - retrieve existing groundings by similarity to topic
class WikiGroundingGenerator:   
    def __init__(self, genai_client, rag_impl):
        self.client = genai_client
        self.rag = rag_impl
        with warnings.catch_warnings():
            warnings.simplefilter("ignore") # suppress beta-warning
            self.splitter = HTMLSemanticPreservingSplitter(
                headers_to_split_on=[("h2", "Main Topic"), ("h3", "Sub Topic")],
                separators=["\n\n", "\n", ". ", "! ", "? "],
                max_chunk_size=2000, # chunk by token limit of models/text-embedding-004
                chunk_overlap=50,
                preserve_links=True,
                preserve_images=True,
                preserve_videos=True,
                preserve_audio=True,
                elements_to_preserve=["table", "ul", "ol", "code"],
                denylist_tags=["script", "style", "head"],
                custom_handlers={"code": self.code_handler},
            )

    def generate_answer(self, query: str, topic: str):
        stored = self.rag.get_wiki_documents(topic)
        if len(stored) > 0:
            return self.rag.generate_with_wiki_passages(query, topic, stored).text
        else:
            pages = wikipedia.search(topic + " company")
            if len(pages) > 0:
                p_topic_match = 0.80
                for i in range(len(pages)):
                    if tqdm(self.similarity(topic, pages[i]) > p_topic_match, 
                            desc= "Score wiki search by similarity to topic"):
                        request = requests.get(f"https://en.wikipedia.org/wiki/{pages[i]}")
                        chunks = [
                            VectorStoreResult(
                                docs=chunk.page_content
                            ) for chunk in self.splitter.split_text(request.text)]
                        self.rag.add_wiki_documents(topic, chunks)
                        return self.rag.generate_with_wiki_passages(query, topic, chunks).text

    def code_handler(self, element: Tag) -> str:
        data_lang = element.get("data-lang")
        code_format = f"<code:{data_lang}>{element.get_text()}</code>"
        return code_format

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def similarity(self, topic: str, page: str):
        return GeminiEmbedFunction(api.client, semantic_mode = True).sts([topic + " company", page])

In [18]:
# An implementation of Search-Grounding Generation.
# - using gemini-2.0-flash with GoogleSearch tool for response generation
# - using a RAG-implementation to store groundings
# - create new groundings by exact match to topic
# - retrieve existing groundings by similarity to topic
class SearchGroundingGenerator:
    config_ground = types.GenerateContentConfig(
        tools=[types.Tool(google_search=types.GoogleSearch())],
        temperature=0.0
    )
    
    def __init__(self, genai_client, rag_impl):
        self.client = genai_client
        self.rag = rag_impl

    def generate_answer(self, query: str, topic: str):
        stored = self.rag.get_grounding_documents(query, topic)
        if len(stored) > 0:
            for i in range(len(stored)):
                doc = stored[i].docs
                meta_q = stored[i].meta["question"]
                p_ground_match = 0.95 # This can be really high ~ 95-97%
                if tqdm(self.similarity(query, meta_q) > p_ground_match,
                        desc="Score similarity to stored grounding"):
                    return ast.literal_eval(doc)[0]["text"]
        return self.get_grounding(query, topic)

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def similarity(self, question: str, compare: str):
        return GeminiEmbedFunction(api.client, semantic_mode = True).sts([question, compare])

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def get_grounding(self, query: str, topic: str):
        contents = [types.Content(role="user", parts=[types.Part(text=query)])]
        contents += f"""
        You're a search assistant that provides grounded answers to questions about {topic}. You will provide only 
        results that discuss {topic}. Be brief and specific in answering and omit extra details.
        If an answer is not possible respond with: I don't know."""
        response = api.retriable(self.client.models.generate_content, 
                                 model=api(Gemini.Model.GEN), 
                                 config=self.config_ground, 
                                 contents=contents)
        if response.candidates[0].grounding_metadata.grounding_supports is not None:
            if topic.replace("'", "") not in response.text: # Exact topic match required
                return Gemini.Const.Stop() # Workaround a bug in gemini-2.0-flash (MGM Studio becomes MGM Resorts)
            else:
                self.rag.add_grounded_document(query, topic, response)
                return response.text
        return Gemini.Const.Stop() # Empty grounding_supports means grounding not possible for query.

In [19]:
# An implementation of Rest-Grounding Generation.
# - using gemini-2.0-flash for response generation
# - using a RAG-implementation to store groundings
# - reduce long-context by chunked pre-processing
class RestGroundingGenerator:
    exchange_codes: Optional[dict] = None
    exchange_lists: dict = {}
    
    def __init__(self, genai_client, rag_impl):
        self.client = genai_client
        self.rag = rag_impl

    def exchange_codes(self) -> dict:
        if self.exchange_codes is None:
            data = self.rag.get_exchanges_csv(
                """Give me a dictionary in string form. It must contain key:value pairs 
                mapping exchange code to name. Just the dictionary string. 
                Omit all other information or details. Do not chat or use sentences.""")
            self.exchange_codes = ast.literal_eval(data.text.strip("\`"))
        return self.exchange_codes

    def exchange_codes(self, with_query: str):
        if with_query not in self.exchange_lists.keys():
            data = self.rag.get_exchanges_csv(
                f"""What is the {with_query} exchange code? Return only the exchange codes 
                as a list in string form. Just the list string. 
                Omit all other information or details. Do not chat or use sentences.""")
            self.exchange_list[with_query] = ast.literal_eval(data.text)
        return self.exchange_lists[with_query]

    def last_market_close(self):
        est = pytz.timezone('US/Eastern') # Exchanges data is in eastern time.
        return self.rag.get_exchanges_csv(
            f"""Answer based on your knowledge of exchange operating hours.
            Do not answer in full sentences. Omit all chat and provide the answer only.
            All exchanges are open on weekdays. Weekdays are: Mon, Tue, Wed, Thu, Fri. Open/Close happens on weekdays.
            All exchanges are closed on weekends. Weekends are: Sat, Sun. No Open/Close happens on weekends.
            The fields pre_market and post_market both represent open hours.
            
            The current date and time is: {datetime.now(est).strftime('%c')}
            
            When was the US exchange's last operating hours? 
            Provide the last weekday's close.
            Include any post-market hours.
            Answer with a date that uses this format: '%a %b %d %X %Y'.""").text

    def get(self, url: str) -> Optional[str]:
        try:
            request = requests.get(url)
            if request.status_code != requests.codes.ok:
                print(f"the endpoint returned status {request.status_code}")
            return request.text
        except Exception as e:
            raise e

    def basemodel(self, data: str, schema: BaseModel, from_lambda: bool = False) -> Optional[BaseModel]:
        try:
            if from_lambda:
                return schema(results=json.loads(data))
            return schema.model_validate_json(data)
        except Exception as e:
            raise e

    def dailycandle(self, data: str) -> Optional[DailyCandle]:
        try:
            date = json.loads(data)["from"]
            agg = self.basemodel(data, Aggregate)
            return DailyCandle(from_date=date, 
                               status=agg.status.value, 
                               symbol=agg.symbol, 
                               open=agg.open, 
                               high=agg.high, 
                               low=agg.low, 
                               close=agg.close, 
                               volume=agg.volume, 
                               otc=agg.otc, 
                               preMarket=agg.preMarket, 
                               afterHours=agg.afterHours)
        except Exception as e:
            raise e

    def try_url(self, url: str, schema: BaseModel, as_lambda: bool, success_fn: Callable, *args, **kwargs):
        try:
            data = self.get(url)
            model = self.basemodel(data, schema, as_lambda)
        except Exception as e:
            return Gemini.Const.Stop()
        else:
            return success_fn(model=model, *args, **kwargs)

    def get_symbol_matches(self, model: SymbolResultFinn, with_id: str, query: str, by_name: bool = True):
        matches = []
        max_failed_match = model.count if not by_name else 3
        p_desc_match = 0.80
        p_symb_match = 0.95
        if model.count > 0:
            for obj in tqdm(model.result, desc="Score similarity to query"):
                if max_failed_match > 0:
                    desc = [with_id.upper(), obj.description.split("-", -1)[0]]
                    symb = [with_id.upper(), obj.symbol]
                    if by_name and similarity(desc) > p_desc_match: 
                        matches.append(obj.symbol)
                    elif not by_name and similarity(symb) > p_symb_match:
                        matches.append(obj.description)
                        max_failed_match = 0
                    else:
                        max_failed_match -= 1
        if len(matches) > 0:
            self.rag.add_api_document(query, matches, with_id, "get_symbol_1")
            return matches
        else:
            return Gemini.Const.Stop()

    def store_quote(self, model: Quote, content) -> Quote:
        self.rag.add_quote_document(content["query"], model.json(), content["symbol"], model.t, "get_quote_1")
        return model

    def load_stored_quotes(self, stored, with_content):
        last_close = parse(self.last_market_close()).timestamp()
        for obj in stored:
            if obj.meta["timestamp"] >= last_close:
                return [json.loads(quote.docs)[0]["answer"] for quote in stored]
        return get_current_price_1(with_content) # update stale quotes

    def store_peers(self, model: PeersResultFinn, with_content):
        symbols = model.results
        if len(symbols) > 0:
            names = []
            for peer in symbols:
                if peer == with_content["symbol"]:
                    continue # skip including the query symbol in peers
                name_query = dict(q=peer, exchange=with_content["exchange"], query=with_content["query"])
                name = get_name_1(name_query)
                if name != Gemini.Const.Stop():
                    data = {"symbol": peer, "name": name}
                    names.append(data)
            peers = {"symbol": with_content["symbol"], "peers": names}
            self.rag.add_peers_document(
                with_content["query"], 
                json.dumps(peers), 
                with_content["symbol"], 
                "get_peers_1", 
                with_content['grouping'])
            return list(peers.items())
        return Gemini.Const.Stop()

    def store_financials(self, model: BasicFinancials, with_content):
        metric = list(model.metric.items())
        chunks = []
        # Chunk the metric data.
        for i in range(0, len(metric), Gemini.Const.MetricBatch()):
            batch = metric[i:i + Gemini.Const.MetricBatch()]
            chunks.append({"question": with_content["query"], "answer": json.dumps(batch)})
        # Chunk the series data.
        for key in model.series.keys():
            series = list(model.series[key].items())
            for s in series:
                if api.token_count(s) <= Gemini.Const.ChunkMax():
                    chunks.append({"question": with_content["query"], "answer": json.dumps(s)})
                else:
                    k = s[0]
                    v = s[1]
                    for i in range(0, len(v), Gemini.Const.SeriesBatch()):
                        batch = v[i:i + Gemini.Const.SeriesBatch()]
                        chunks.append({"question": with_content["query"], "answer": json.dumps({k: batch})})
        self.rag.add_rest_chunks(with_content["query"], chunks, with_content["symbol"], "get_financials_1")
        return chunks

# Testing the RAG Implementation

<span style="font-size:18px;">
Let's load some test data and see what the RAG can do. The test data is a CSV file containing stock market exchange data. It includes the market id code, name, locale, and operating hours. The import will use CSVLoader from <code>langchain-community</code> to parse the exchange data into Documents that our RAG can ingest.
</span>

In [20]:
# Load the exchange data from source csv.
# - Identifies exchanges by a 1-2 letter code which can be used to filter response data.
# - Also maps the exchange code to exchange details.
df = pandas.read_csv("/kaggle/input/exchanges/exchanges_src.csv").drop(["close_date"], axis=1).fillna("")
df.to_csv("exchanges.csv", index=False)
exchanges = CSVLoader(file_path="exchanges.csv", encoding="utf-8", csv_args={"delimiter": ","}).load()

# Prepare a RAG tool for use and add the exchange data.
tool_rag = RetrievalAugmentedGenerator(api.client, "finance")
tool_rag.add_documents_list(exchanges)

# Prepare a the grounding tools for use.
tool_wiki = WikiGroundingGenerator(api.client, tool_rag)
tool_ground = SearchGroundingGenerator(api.client, tool_rag)
tool_rest = RestGroundingGenerator(api.client, tool_rag)

Generate document embedding: 0it [00:00, ?it/s]


<span style="font-size:18px;">
Now that the data is loaded lets ask our RAG to perform some augmenting. We can ask it to perform all sorts of useful tasks. We'll generate some useful reusable data structures and check to make sure it can answer important questions. The exchanges all have id's which are used to filter the realtime data. So we'll make sure the RAG know how to create this mapping. We'll also check it's awareness of operating hours. After all, Essy, doesn't mindlessly hammer away at api's when no new data is available.
</span>

In [21]:
# The RAG tool is a helpful expert.

response = tool_rag.get_exchanges_csv(
    """Give me a dictionary in string form. It must contain key:value pairs mapping 
    exchange code to name. Just the dictionary string in pretty form.""")
print(response.text)

response = tool_rag.get_exchanges_csv(
    """What is the Germany exchange code? Return only the exchange codes as a simple 
    comma separated value that I can copy.""")
print(response.text, "\n")

response = tool_rag.get_exchanges_csv("What are the Germany exchanges and thier corresponding exchange codes?")
print(response.text, "\n")

response = tool_rag.generate_answer("What are Google's stock ticker symbols?")
print(response.text)

response = tool_rag.get_exchanges_csv("What are the US exchange operating hours?")
print(response.text, "\n")

est = pytz.timezone('US/Eastern') # Exchanges data is in eastern time.
response = tool_rag.get_exchanges_csv(
    f"""Answer based on your knowledge of exchange operating hours. 
    Do not answer in full sentences. Omit all chat and provide the answer only. 
    All exchanges are open on weekdays. Weekdays are: Mon, Tue, Wed, Thu, Fri. Open/Close happens on weekdays. 
    All exchanges are closed on weekends. Weekends are: Sat, Sun. No Open/Close happens on weekends. 
    The fields pre_market and post_market both represent open hours.
    
    The current date and time is: {datetime.now(est).strftime('%c')}
    
    When was the US exchange's last operating hours? Provide the last weekday's close. Include any post-market hours. 
    Answer with a date that uses this format: '%a %b %d %X %Y'.""")
print(response.text)

```
{
    "SC": "BOERSE_FRANKFURT_ZERTIFIKATE",
    "SX": "DEUTSCHE BOERSE Stoxx",
    "HK": "HONG KONG EXCHANGES AND CLEARING LTD",
    "DB": "DUBAI FINANCIAL MARKET",
    "NZ": "NEW ZEALAND EXCHANGE LTD",
    "QA": "QATAR EXCHANGE",
    "KS": "KOREA EXCHANGE (STOCK MARKET)",
    "SW": "SWISS EXCHANGE",
    "DU": "BOERSE DUESSELDORF",
    "BC": "BOLSA DE VALORES DE COLOMBIA",
    "KQ": "KOREA EXCHANGE (KOSDAQ)",
    "SN": "SANTIAGO STOCK EXCHANGE",
    "SI": "SINGAPORE EXCHANGE",
    "AD": "ABU DHABI SECURITIES EXCHANGE",
    "CO": "OMX NORDIC EXCHANGE COPENHAGEN A/S",
    "L": "LONDON STOCK EXCHANGE",
    "ME": "MOSCOW EXCHANGE",
    "TO": "TORONTO STOCK EXCHANGE",
    "BD": "BUDAPEST STOCK EXCHANGE",
    "TG": "DEUTSCHE BOERSE TradeGate",
    "US": "US exchanges (NYSE, Nasdaq)",
    "TW": "TAIWAN STOCK EXCHANGE",
    "JK": "INDONESIA STOCK EXCHANGE",
    "SZ": "SHENZHEN STOCK EXCHANGE",
    "VS": "NASDAQ OMX VILNIUS",
    "MX": "BOLSA MEXICANA DE VALORES (MEXICAN STOCK EXCHANGE)",
 

<span style="font-size:18px;">
Excellent! Though, despite my best effort I could not convince Gemini to apply date correction (during chaining) based on holiday. It simply wasn't stable enough to be useful. I would either have to add a holiday data set, or (what I chose) apply a quick temporary fix. A real-time API endpoint may fail due to a holiday being selected as the date. If that happens I'll just retry Thursday if the failure happened on Friday, likewise choosing Friday if the failure happened on Monday. Crude but simple for foundational purposes.
</span>

# Declaring the Function Calling Metadata

<span style="font-size:18px;">
Our Function Calling expert will chain together the other experts we've implemented thus far. It also provides the final response through augmentation. This time using the tools as a source of grounding truth. It'd like to say it's all truth organised by topic and other metadata. It's still a precarious situation if Essy incidently chains into mining data on another topic. We want Amazon to be the owner of MGM Studio's not MGM Resorts International. We also don't want a summary to include another company unless that company is a peer.
</span>

<span style="font-size:18px;">
The function calling metadata is thus extremely important. It needs to combine our other experts with the real-time api's data. Essy will use two API providers as sources of finance data. The primary motivation being that each provider has limits in their own way, yet both are useful in their own own way. This is useful anywhere you need a broad spectrum of sources of truth. At metadata creation I'll adopt the naming convention of appending the provider (if any) id. This helps keep functions more understandable when you know which provider you're dealing with.
</span>

In [22]:
# Declare callable functions using OpenAPI schema.
decl_get_symbol_1 = types.FunctionDeclaration(
    name="get_symbol_1",
    description="""Search for the stock ticker symbol of a given company, security, isin or cusip. Each ticker
                   entry provides a description, symbol, and asset type. If this doesn't help you should try 
                   calling get_wiki_tool_response next.""",
    parameters={
        "type": "object",
        "properties": {
            "q": {
                "type": "string",
                "description": """The company, security, isin or cusip to search for a symbol."""
            },
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["q", "exchange", "query"]
    }
)

decl_get_symbols_1 = types.FunctionDeclaration(
    name="get_symbols_1",
    description="""List all supported symbols and tickers. The results are filtered by exchange code.""",
    parameters={
        "type": "object",
        "properties": {
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter the results."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["exchange", "query"]
    }
)

decl_get_name_1 = types.FunctionDeclaration(
    name="get_name_1",
    description="""Search for the name associated with a stock ticker or symbol's company, security, isin or cusip. 
    Each ticker entry provides a description, matching symbol, and asset type.""",
    parameters={
        "type": "object",
        "properties": {
            "q": {
                "type": "string",
                "description": """The symbol or ticker to search for."""
            },
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            },
            "company": {
                "type": "string",
                "description": "The company you're searching for."
            }
        },
        "required": ["q", "exchange", "query", "company"]
    }
)

decl_get_symbol_quote_1 = types.FunctionDeclaration(
    name="get_symbol_quote_1",
    description="""Search for the current price or quote of a stock ticker or symbol. The response is
                   provided in json format. Each response contains the following key-value pairs:
                   
                   c: Current price,
                   d: Change,
                  dp: Percent change,
                   h: High price of the day,
                   l: Low price of the day,
                   o: Open price of the day,
                  pc: Previous close price,
                   t: Epoch timestamp of price in seconds.

                   Parse the response and respond according to this information.""",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "The stock ticker symbol for a company, security, isin, or cusip." 
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            },
            "exchange": {
                "type": "string",
                "description": "The exchange code used to filter quotes. This must always be 'US'."
            }
        },
        "required": ["symbol", "query", "exchange"]
    }
)

decl_get_local_datetime = types.FunctionDeclaration(
    name="get_local_datetime",
    description="""Converts an array of timestamps from epoch time to the local timezone format. The result is an array
                   of date and time in locale appropriate format. Suitable for use in a locale appropriate response.
                   Treat this function as a vector function. Always prefer to batch timestamps for conversion. Use this
                   function to format date and time in your responses.""",
    parameters={
        "type": "object",
        "properties": {
            "t": {
                "type": "array",
                "description": """An array of timestamps in seconds since epoch to be converted. The order of
                                  timestamps matches the order of conversion.""",
                "items": {
                    "type": "integer"
                }
            }
        },
        "required": ["t"]
    }
)

decl_get_market_status_1 = types.FunctionDeclaration(
    name="get_market_status_1",
    description="""Get the current market status of global exchanges. Includes whether exchanges are open or closed.  
                   Also includes holiday details if applicable. The response is provided in json format. Each response 
                   contains the following key-value pairs:

                   exchange: Exchange code,
                   timezone: Timezone of the exchange,
                    holiday: Holiday event name, or null if it's not a holiday,
                     isOpen: Whether the market is open at the moment,
                          t: Epoch timestamp of status in seconds (Eastern Time),
                    session: The market session can be 1 of the following values: 
                    
                    pre-market,regular,post-market when open, or null if closed.
                    
                    Parse the response and respond according to this information.""",
    parameters={
        "type": "object",
        "properties": {
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. The default if omitted is 'US' for the 
                                  US exchanges. A dictionary mapping supported exchange codes (key) to their 
                                  description (value) can be obtained from get_exchange_codes_1. Search the values for
                                  a matching exchange code if unsure."""
            }
        },
        "required": ["exchange"]
    }
)

decl_get_company_peers_1 = types.FunctionDeclaration(
    name="get_company_peers_1",
    description="""Search for a company's peers. Returns a list of peers operating in the same country and in the same
                   sector, industry, or subIndustry. Each response contains the following key-value pairs: 
                   
                   symbol: The company's stock ticker symbol, 
                   peers: A list containing the peers.
                   
                   Each peers entry contains the following key-value pairs:
                   
                   symbol: The peer company's stock ticker symbol, 
                   name: The peer company's name.
                   
                   Parse the response and respond according to this information.""",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "The stock ticker symbol of a company to obtain peers."
            },
            "grouping": {
                "type": "string",
                "description": """Specify the grouping category for choosing peers. When not specified the default
                                  category is subIndustry. This parameter may be one of the following values: 
                                  sector, industry, subIndustry."""
            },
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["symbol", "grouping", "exchange", "query"]
    }
)

decl_get_exchange_codes_1 = types.FunctionDeclaration(
    name="get_exchange_codes_1",
    description="""Get a dictionary mapping all supported exchange codes to their names."""
)

decl_get_exchange_code_1 = types.FunctionDeclaration(
    name="get_exchange_code_1",
    description="""Search for the exchange code to use when filtering by exchange. The result will be one or
                   more exchange codes provided as a comma-separated string value.""",
    parameters={
        "type": "object",
        "properties": {
            "q": {
                "type": "string",
                "description": "Specifies which exchange code to search for."
            }
        },
        "required": ["q"]
    }
)

decl_get_financials_1 = types.FunctionDeclaration(
    name="get_financials_1",
    description="""Get company basic financials such as margin, P/E ratio, 52-week high/low, etc. Parse the response for 
                   key-value pairs in json format and interpret their meaning as stock market financial indicators.""",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "Stock ticker symbol for a company."
            },
            "metric": {
                "type": "string",
                "description": "It must always be declared as the value 'all'"
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["symbol", "metric", "query"]
    }
)

decl_get_daily_candlestick_2 = types.FunctionDeclaration(
    name="get_daily_candlestick_2",
    description="""Get a historical daily stock ticker candlestick / aggregate bar (OHLC). 
                   Includes historical daily open, high, low, and close prices. Also includes historical daily trade
                   volume and pre-market/after-hours trade prices. It does not provide today's data until after 
                   11:59PM Eastern Time.""",
    parameters={
        "type": "object",
        "properties": {
            "stocksTicker": {
                "type": "string",
                "description": "The stock ticker symbol of a company to search for.",
            },
            "date": {
                "type": "string",
                "format": "date-time",
                "description": """The date of the requested candlestick in format YYYY-MM-DD. The default is one 
                                  weekday prior to get_last_market_close (excluding weekends). This date must never 
                                  be more recent than the default. Replace more recent dates with the default."""
            },
            "adjusted": {
                "type": "string",
                "description": """May be true or false. Indicated whether or not the results are adjusted for splits. 
                                  By default, results are adjusted. Set this to false to get results that are NOT 
                                  adjusted for splits."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["stocksTicker", "date", "adjusted", "query"]
    },
)

decl_get_company_news_1 = types.FunctionDeclaration(
    name="get_company_news_1",
    description="Retrieve the most recent news articles related to a specified ticker.",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "Stock ticker symbol for a company.",
            },
            "from": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD. It must be older than the parameter 'to'. The default
                                  value is one month ago."""
            },
            "to": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD. It must be more recent than the parameter 'from'. The
                                  default value is today's date."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["symbol", "from", "to", "query"]
    },
)

decl_get_custom_candlestick_2 = types.FunctionDeclaration(
    name="get_custom_candlestick_2",
    description="""Get a historical stock ticker candlestick / aggregate bar (OHLC) over a custom date range and 
                   time interval in Eastern Time. Includes historical open, high, low, and close prices. Also 
                   includes historical daily trade volume and pre-market/after-hours trade prices. It does not
                   include today's open, high, low, or close until after 11:59PM Eastern Time.""",
    parameters={
        "type": "object",
        "properties": {
            "stocksTicker": {
                "type": "string",
                "description": "The stock ticker symbol of a company to search for.",
            },
            "multiplier": {
                "type": "integer",
                "description": "This must be 1 unless told otherwise."
            },
            "timespan": {
                "type": "string",
                "description": """The size of the candlestick's time window. This is allowed to be one of the following:
                                  second, minute, hour, day, week, month, quarter, or year. The default value is day."""
            },
            "from": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD must be older than the parameter 'to'. The default
                                  value is one-month ago from today's date."""
            },
            "to": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD must be more recent than the parameter 'from'. The 
                                  default is one weekday prior to get_last_market_close (excluding weekends).
                                  Replace more recent dates with the default."""
            },
            "adjusted": {
                "type": "string",
                "description": """May be true or false. Indicated whether or not the results are adjusted for splits. 
                                  By default, results are adjusted. Set this to false to get results that are NOT 
                                  adjusted for splits."""
            },
            "sort": {
                "type": "string",
                "description": """May be one of asc or desc. asc will sort by timestmap in ascending order. desc will
                                  sort by timestamp in descending order."""
            },
            "limit": {
                "type": "integer",
                "description": """Set the number of base aggregates used to create this candlestick. This must be 5000 
                                  unless told to limit base aggregates to something else."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["stocksTicker", "multiplier", "timespan", "from", "to", "adjusted", "sort", "limit", "query"]
    },
)

decl_get_last_market_close = types.FunctionDeclaration(
    name="get_last_market_close",
    description="""Get the date and time of the US exchange market's last close. Provides the last US market close in 
                   locale appropriate format."""
)

decl_get_ticker_overview_2 = types.FunctionDeclaration(
    name="get_ticker_overview_2",
    description="""Retrieve comprehensive details for a single ticker symbol. It's a deep look into a company’s 
    fundamental attributes, including its primary exchange, standardized identifiers (CIK, composite FIGI, 
    share class FIGI), market capitalization, industry classification, and key dates. Also includes branding assets in
    the form of icons and logos.
    """,
    parameters={
        "type": "object",
        "properties": {
            "ticker": {
                "type": "string",
                "description": "Stock ticker symbol of a company."
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["ticker", "query"]
    }
)

decl_get_recommendation_trends_1 = types.FunctionDeclaration(
    name="get_recommendation_trends_1",
    description="""Get the latest analyst recommendation trends for a company.
                The data includes the latest recommendations as well as historical
                recommendation data for each month. The data is classified according
                to these categories: strongBuy, buy, hold, sell, and strongSell.
                The date of a recommendation indicated by the value of 'period'.""",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "Stock ticker symbol for a company."
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["symbol", "query"]
    }
)

decl_get_news_with_sentiment_2 = types.FunctionDeclaration(
    name="get_news_with_sentiment_2",
    description="""Retrieve the most recent news articles related to a specified ticker. Each article includes 
                   comprehensive coverage. Including a summary, publisher information, article metadata, 
                   and sentiment analysis.""",
    parameters={
        "type": "object",
        "properties": {
            "ticker": {
                "type": "string",
                "description": "Stock ticker symbol for a company."
            },
            "published_utc": {
                "type": "string",
                "format": "date-time",
                "description": """Omit this parameter unless you're told told to filter by published_utc."""
            },
            "order": {
                "type": "string",
                "description": """Must be desc for descending order, or asc for ascending order.
                                  When order is not specified the default is descending order.
                                  Ordering will be based on the parameter: sort."""
            },
            "limit": {
                "type": "integer",
                "description": """This must be 100 unless told to limit news results to something else."""
            },
            "sort": {
                "type": "string",
                "description": """The sort field used for ordering. This value must
                                  always be published_utc."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["ticker", "order", "limit", "sort", "query"]
    }
)

decl_get_rag_tool_response = types.FunctionDeclaration(
    name="get_rag_tool_response",
    description="""A database containing useful financial information. Always check here for answers first.""",
    parameters={
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "A question needing an answer. Asked as a simple string."
            }
        }
    }
)

decl_get_wiki_tool_response = types.FunctionDeclaration(
    name="get_wiki_tool_response",
    description="""Answers questions that still have unknown answers. Retrieve a wiki page related to a company, 
                   product, or service. Each web page includes detailed company information, financial indicators, 
                   tickers, symbols, history, and products and services.""",
    parameters={
        "type": "object",
        "properties": {
            "id": {
                "type": "string",
                "description": "The question's company or product. Just the name and no other details."
            },
            "q": {
                "type": "string",
                "description": "The complete, unaltered, query string."
            }
        },
        "required": ["id", "q"]
    }
)

decl_get_search_tool_response = types.FunctionDeclaration(
    name="get_search_tool_response",
    description="Answers questions that still have unknown answers. Use it after checking all your other tools.",
    parameters={
        "type": "object",
        "properties": {
            "q": {
                "type": "string",
                "description": "The question needing an answer. Asked as a simple string."
            },
            "id": {
                "type": "string",
                "description": "The question's company or product. In one word. Just the name and no other details."
            }
        },
        "required": ["q", "id"]
    }
)

# Implementing the Function Calls

<span style="font-size:18px;">
One downside of this part being the main part was the lack of time to refactor this part more. Our formative Essy implements as much useful data from two finacial APIs. In order to use it you will need to declare secrets for <a class="anchor-link" href="https://finnhub.io/dashboard">Finnhub</a> and <a class="anchor-link" href="https://polygon.io/dashboard">Polygon</a> finance APIs. Register at their respective sites for your free API key. Then import the secret using the same method as how you setup Google's API key.
</span>

In [23]:
# Implement the callable functions and the function handler.

def ask_rag_tool(content):
    return tool_rag.generate_answer(content["question"]).text

def ask_wiki_tool(content):
    return tool_wiki.generate_answer(content["q"], content["id"])

def ask_search_tool(content):
    return tool_ground.generate_answer(content["q"], content["id"])

def get_exchange_codes_1(content):
    return tool_rest.exchange_codes()

def get_exchange_code_1(content):
    return tool_rest.exchange_codes(with_query=content)

def last_market_close(content):
    return tool_rest.last_market_close()

@retry.Retry(
    predicate=is_retriable,
    initial=2.0,
    maximum=64.0,
    multiplier=2.0,
    timeout=600,
)
def similarity(content):
    return GeminiEmbedFunction(api.client, semantic_mode = True).sts(content)
    
def get_symbol_1(content, by_name: bool = True):
    stored = tool_rag.get_api_documents(content["query"], content["q"], "get_symbol_1")
    if len(stored) == 0:
        return tool_rest.try_url(
            f"https://finnhub.io/api/v1/search?q={content['q']}&exchange={content['exchange']}&token={FINNHUB_API_KEY}",
            schema=SymbolResultFinn,
            as_lambda=False,
            success_fn=tool_rest.get_symbol_matches,
            with_id=content['q'],
            query=content["query"],
            by_name=by_name)
    return json.loads(stored[0].docs)["answer"]

def get_symbols_1(content):
    return None # todo

def get_name_1(content):
    return get_symbol_1(content, by_name = False)

def get_quote_1(content):
    stored = tool_rag.get_api_documents(content["query"], content["symbol"], "get_quote_1")
    isOpen = tool_rest.basemodel(get_market_status_1(content), schema=MarketStatus).isOpen
    if len(stored) == 0 or isOpen:
        return get_current_price_1(content)
    else:
        return tool_rest.load_stored_quotes(stored, with_content=content)

def get_current_price_1(content):
    return tool_rest.try_url(
        f"https://finnhub.io/api/v1/quote?symbol={content['symbol']}&token={FINNHUB_API_KEY}",
        schema=Quote,
        as_lambda=False,
        success_fn=tool_rest.store_quote, 
        content=content).json()

def get_market_status_1(content):
    return tool_rest.try_url(
        f"https://finnhub.io/api/v1/stock/market-status?exchange={content['exchange']}&token={FINNHUB_API_KEY}",
        schema=MarketStatusResult, 
        as_lambda=True, 
        success_fn=lambda model: model.results).json()

def get_peers_1(content):
    stored = tool_rag.get_peers_document(content["query"], content["symbol"], content['grouping'])
    if len(stored) == 0:
        return tool_rest.try_url(
            f"https://finnhub.io/api/v1/stock/peers?symbol={content['symbol']}&grouping={content['grouping']}&token={FINNHUB_API_KEY}",
            schema=PeersResultFinn,
            as_lambda=True,
            success_fn=tool_rest.store_peers, 
            with_content=content)
    else:
        return ast.literal_eval(stored[0].docs)[0]["answer"] # The first document should be most relevant.

def local_datetime(content):
    local_t = []
    for timestamp in content["t"]:
        local_t.append(local_date_from_epoch(timestamp))
    return local_t

def local_date_from_epoch(timestamp):
    est = pytz.timezone('US/Eastern') # Exchanges data is in eastern time.
    if len(str(timestamp)) == 13:
        return datetime.fromtimestamp(timestamp/1000, tz=est).strftime('%c')
    else:
        return datetime.fromtimestamp(timestamp, tz=est).strftime('%c')

def get_financials_1(content):
    stored = tool_rag.get_basic_financials(content["query"], content["symbol"], "get_financials_1")
    if len(stored) == 0:
        return tool_rest.try_url(
            f"https://finnhub.io/api/v1/stock/metric?symbol={content['symbol']}&metric={content['metric']}&token={FINNHUB_API_KEY}",
            schema=BasicFinancials,
            as_lambda=False,
            success_fn=tool_rest.store_financials,
            with_content=content)
    return [chunk.docs for chunk in stored]

def get_news_1(content):
    #news = tool_rag.get_api_documents(content["query"], content["symbol"], "get_news_1")
    #if len(news[0]) == 0:
        url = f"https://finnhub.io/api/v1/company-news?symbol={content['symbol']}&from={content['from']}&to={content['to']}&token={FINNHUB_API_KEY}"
        try:
            news = json.loads(requests.get(url).text)
        except:
            return Gemini.Const.Stop()
        else:
            if len(news) == 0:
                return Gemini.Const.Stop()
            #tool_rag.add_api_document(content["query"], news, content["symbol"], "get_news_1")
            return list(news.items())
    #return news

def get_daily_candle_2(content):
    daily_candle = tool_rag.get_api_documents(content["query"], content["stocksTicker"], "daily_candle_2")
    if len(daily_candle[0]) == 0:
        url = f"https://api.polygon.io/v1/open-close/{content['stocksTicker']}/{content['date']}?adjusted={content['adjusted']}&apiKey={POLYGON_API_KEY}"
        try:
            request = requests.get(url)
            daily_candle = ast.literal_eval(request.text)
        except:
            return f"I don't know. Endpoint returned status {request.status_code}"
        else:
            if daily_candle["status"] in ["OK","DELAYED"]:
                tool_rag.add_api_document(content["query"], daily_candle, content["stocksTicker"], "daily_candle_2")
                return list(daily_candle.items())
            else:
                date = parse(content["date"])
                new_date = None
                if date.weekday() == 4: # index 4 for friday
                    new_date = date - timedelta(days=1)
                elif date.weekday() == 0: # index 0 for monday
                    new_date = date - timedelta(days=3)
                if new_date is None:
                    return Gemini.Const.Stop()
                content["date"] = new_date.strftime("%Y-%m-%d")
                return get_daily_candle_2(content)
    return daily_candle

def get_custom_candle_2(content):
    url = f"""https://api.polygon.io/v2/aggs/ticker/{content['stocksTicker']}/range/{content['multiplier']}/{content['timespan']}/{content['from']}/{content['to']}?adjusted={content['adjusted']}&sort={content['sort']}&limit={content['limit']}&apiKey={POLYGON_API_KEY}"""
    try:
        request = requests.get(url)
        custom_candle = json.loads(request.text)
    except:
        return f"I don't know. Endpoint returned status {request.status_code}"
    else:
        if custom_candle["status"] in ["OK","DELAYED"]:
            tool_rag.add_api_document(content["query"], custom_candle, content["stocksTicker"], "custom_candle_2")
            return list(custom_candle.items())
        return Gemini.Const.Stop()

def get_overview_2(content):
    overview = tool_rag.get_api_documents(content["query"], content["ticker"], "ticker_overview_2")
    if len(overview[0]) == 0:
        url = f"https://api.polygon.io/v3/reference/tickers/{content['ticker']}?apiKey={POLYGON_API_KEY}"
        try:
            request = requests.get(url)
            overview = json.loads(request.text)
        except:
            return f"I don't know. Endpoint returned status {request.status_code}"
        else:
            if overview["status"] in ["OK","DELAYED"]:
                tool_rag.add_api_document(content["query"], overview, content["ticker"], "ticker_overview_2")
                return list(overview.items())
            return Gemini.Const.Stop()
    return overview

def get_trends_1(content):
    trends = tool_rag.get_api_documents(content["query"], content["symbol"], "trends_1")
    if len(trends[0]) == 0:
        url = f"https://finnhub.io/api/v1/stock/recommendation?symbol={content['symbol']}&token={FINNHUB_API_KEY}"
        try:
            trends = json.loads(requests.get(url).text)
        except:
            return Gemini.Const.Stop()
        else:
            if len(trends) > 0:
                tool_rag.add_api_document(content["query"], trends, content["symbol"], "trends_1")
                return trends
            return Gemini.Const.Stop()
    return trends

def get_news_2(content):
    #news = tool_rag.get_api_documents(content["query"], content["ticker"], "get_news_2")
    #if len(news[0]) == 0:
        url = f"https://api.polygon.io/v2/reference/news?ticker={content['ticker']}&order={content['order']}&limit={content['limit']}&sort={content['sort']}&apiKey={POLYGON_API_KEY}"
        try:
            request = requests.get(url)
            news = json.loads(request.text)
        except:
            return f"I don't know. Endpoint returned status {request.status_code}"
        else:
            if news["status"] in ["OK","DELAYED"]:
                #tool_rag.add_api_document(content["query"], news, content["ticker"], "get_news_2")
                return list(news.items())
            return Gemini.Const.Stop()
    #return news
        
finance_tool = types.Tool(
    function_declarations=[
        decl_get_symbol_1,
        decl_get_symbols_1,
        decl_get_name_1,
        decl_get_symbol_quote_1,
        decl_get_market_status_1,
        decl_get_company_peers_1,
        decl_get_local_datetime,
        decl_get_last_market_close,
        decl_get_exchange_codes_1,
        decl_get_exchange_code_1,
        decl_get_financials_1,
        decl_get_daily_candlestick_2,
        decl_get_custom_candlestick_2,
        decl_get_ticker_overview_2,
        decl_get_recommendation_trends_1,
        decl_get_news_with_sentiment_2,
        decl_get_rag_tool_response,
        decl_get_wiki_tool_response,
        decl_get_search_tool_response
    ]
)

function_handler = {
    "get_symbol_1": get_symbol_1,
    "get_symbols_1": get_symbols_1,
    "get_name_1": get_name_1,
    "get_symbol_quote_1": get_quote_1,
    "get_market_status_1": get_market_status_1,
    "get_company_peers_1": get_peers_1,
    "get_local_datetime": local_datetime,
    "get_last_market_close": last_market_close,
    "get_exchange_codes_1": get_exchange_codes_1,
    "get_exchange_code_1": get_exchange_code_1,
    "get_financials_1": get_financials_1,
    "get_daily_candlestick_2": get_daily_candle_2,
    "get_custom_candlestick_2": get_custom_candle_2,
    "get_ticker_overview_2": get_overview_2,
    "get_recommendation_trends_1": get_trends_1,
    "get_news_with_sentiment_2": get_news_2,
    "get_rag_tool_response": ask_rag_tool,
    "get_wiki_tool_response": ask_wiki_tool,
    "get_search_tool_response": ask_search_tool
}

In [24]:
# Define the system prompt.

instruction = f"""You are a helpful and informative bot that answers finance and stock market questions. 
Only answer the question asked and do not change topic. While the answer is still
unknown you must follow these rules for predicting function call order:

RULE#1: Always consult your other functions before get_search_tool_response.
RULE#2: Always consult get_wiki_tool_response before get_search_tool_response.
RULE#3: Always consult get_search_tool_response last.
RULE#4: Always convert timestamps with get_local_datetime and use the converted date/time in your response.
RULE#5: Always incorporate as much useful information from tools and functions in your response."""

In [25]:
# Import the finance api secret keys.

POLYGON_API_KEY = UserSecretsClient().get_secret("POLYGON_API_KEY")
FINNHUB_API_KEY = UserSecretsClient().get_secret("FINNHUB_API_KEY")

In [26]:
# Implement the function calling expert.

@retry.Retry(
    predicate=is_retriable,
    initial=2.0,
    maximum=64.0,
    multiplier=2.0,
    timeout=600,
)
def send_message(prompt):
    #display(Markdown("#### Prompt"))
    #print(prompt, "\n")
    # Define the user prompt part.
    contents = [types.Content(role="user", parts=[types.Part(text=prompt)])]
    # Gemini's innate notion of current date and time is unstable.
    est = pytz.timezone('US/Eastern') # The finance api data is in eastern time.
    contents += f"""
    The current date and time is: {datetime.now(est).strftime('%c')}
    
    Give a concise, and detailed summary. Use information that you learn from the API responses.
    Use your tools and function calls according to the rules. Convert any all-upper case identifiers
    to proper case in your response. Convert any abbreviated or shortened identifiers to their full forms.
    Convert timestamps according to the rules before including them.
    """
    # Enable system prompt, function calling and minimum-randomness.
    config_fncall = types.GenerateContentConfig(
        system_instruction=instruction,
        tools=[finance_tool],
        temperature=0.0
    )
    # Handle cases with multiple chained function calls.
    function_calling_in_process = True
    while function_calling_in_process:
        # Send the user prompt and function declarations.
        response = api.retriable(api.client.models.generate_content, 
                                 model=api(Gemini.Model.GEN), 
                                 config=config_fncall, 
                                 contents=contents)
        # A part can be a function call or natural language response.
        for part in response.candidates[0].content.parts:
            if function_call := part.function_call:
                # Extract the function call.
                fn_name = function_call.name
                #display(Markdown("#### Predicted function name"))
                #print(fn_name, "\n")
                # Extract the function call arguments.
                fn_args = {key: value for key, value in function_call.args.items()}
                #display(Markdown("#### Predicted function arguments"))
                #print(fn_args, "\n")
                # Call the predicted function.
                api_response = function_handler[fn_name](fn_args)[:20000] # Stay within the input token limit
                #display(Markdown("#### API response"))
                #print(api_response[:500], "...", "\n")
                # Create an API response part.
                api_response_part = types.Part.from_function_response(
                    name=fn_name,
                    response={"content": api_response},
                )
                # Append the model's function call part.
                contents.append(types.Content(role="model", parts=[types.Part(function_call=function_call)])) 
                # Append the api response part.
                contents.append(types.Content(role="user", parts=[api_response_part]))
            else:
                # The model gave a natural language response
                function_calling_in_process = False
                break # No more parts in response.
        if not function_calling_in_process:
            break # The function calling chain is complete.
            
    # Show the final natural language summary
    display(Markdown("#### Natural language response"))
    display(Markdown(response.text.replace("$", "\\\\$")))

# Ask a question

In [27]:
send_message("is the US exchange open?") # basemodel

#### Natural language response

As of May 13, 2025 at 4:52 AM Eastern Time, the US exchange is closed and it is not a holiday.


In [28]:
send_message("what is Apple's stock ticker?") # basemodel

Score similarity to query: 100%|██████████| 17/17 [00:12<00:00,  1.41it/s]
Generate api embedding: 0it [00:00, ?it/s]


#### Natural language response

The stock ticker symbol for Apple is AAPL.


In [29]:
send_message("What is the current price of Amazon stock?") # basemodel

Generate quote embedding: 0it [00:00, ?it/s]


#### Natural language response

The current price of Amazon (AMZN) is \\$208.64. The price changed by \\$15.58, which is a percentage change of 8.07%. The high price of the day was \\$211.66, and the low price of the day was \\$205.75. The opening price of the day was \\$210.71, and the previous close price was \\$193.06. The price was last updated on Mon May 12 2025 at 16:00:00.


In [30]:
send_message("Tell me about apple's basic financials")

Generate chunks embedding: 0it [00:00, ?it/s]


#### Natural language response

Here's a summary of Apple's basic financials:

**Financial Highlights:**

*   **Revenue:** Apple's revenue per employee (TTM) is \\$2.4413 million.
*   **Profitability:**
    *   The gross margin (TTM) is 46.63%.
    *   The operating margin (TTM) is 31.81%.
    *   The net profit margin (TTM) is 24.3%.
*   **Earnings Per Share (EPS):**
    *   The annual earnings per share is \\$6.0836.
    *   The trailing twelve months earnings per share is \\$6.4078.
*   **Dividends:**
    *   The current dividend yield (TTM) is 0.4877%.
    *   The dividend per share (TTM) is \\$1.0115.
*   **Valuation:**
    *   The price-to-earnings ratio (TTM) is 32.2714.
    *   The price-to-book ratio is 47.0059.
    *   The price-to-sales ratio (TTM) is 7.8423.
*   **Balance Sheet:**
    *   The book value per share (Annual) is 3.7673.
    *   The current ratio (Annual) is 0.8673.
    *   The quick ratio (Annual) is 0.826.
*   **Debt:** The long-term debt to equity (Annual) is 1.5057.
*   **Returns:**
    *   The return on assets (TTM) is 28.37%.
    *   The return on equity (TTM) is 151.31%.
    *   The return on invested capital (TTM) is 58.95%.
*   **Volume:** The 10-day average trading volume is 58.70509.
*   **52 Week High:** The 52 week high was 260.1 on 2024-12-26.
*   **52 Week Low:** The 52 week low was 169.2101 on 2025-04-08.
*   **Market Capitalization:** The market capitalization is 3139809.2.
*   **Enterprise Value:** The enterprise value is 3209833.2.


In [ ]:
send_message(
    """Tell me Amazon's current share price and provide candlestick data for the past month.
    Sort the data in descending order by date. Format the prices consistently as currency.
    Round prices to two decimal places.
    Present the data with multiple columns for display in markdown.""")

In [ ]:
send_message(
    '''Tell me about Amazon's current bullish versus bearish predictions, and recommendation trends.
    Include a discussion of any short-term trends, and sentiment analysis.''')

In [ ]:
send_message("What is Google's stock ticker symbol?")

In [ ]:
send_message(
    '''Tell me about Google's share price over the past month.
    Perform a sentiment analysis of news during the same period. Include trends.''')

In [ ]:
send_message("What is MGM Studio's stock symbol?")

In [ ]:
send_message("What is Amazon MGM Studio's stock symbol?")

In [ ]:
send_message("What is Facebook's stock ticker symbol?")

In [ ]:
send_message(
    '''How is the outlook for Apple based on trends and news sentiment over the past month?
    Perform the same analysis on Apple's peers. Then compare Apple result to it's peers.''')

In [ ]:
send_message(
    '''What does the recent news say about Apple and the impact of tariffs? Over the past 2 month.
    Avoid discussing duplicate news entries.''')

# Conclusion

<span style="font-size:18px;">
For now that will have to do. Our Essy has a solid foundation but more could be done to organise metadata. No evaluation or validation has been performed (except fuzzing the prompt). Next steps include restructuring the vector database based on lessons learned. That'll be followed by plotting, multi-modal, and structured output. The last close date (generative) function can be temperamental. In the same way Gemini always feels regarding dates. I've learnt so much. I'm happy I decided to participate in the event! It really has been a joy to see Essy grow from random chat with Gemini into the foundation for a good-broker buddy. I hope you enjoy playing with this edition as much as I enjoyed building it!
</span>